## Shioaji_CNN - 2 (CNN Model for Intraday Trading)

### 本範例程式利用永豐python api取得盤中即時報價並丟入已訓練好的CNN模型中，輸出即時預測結果

需要有永豐金證券的帳號以及憑證才能使用Shioaji API進行下單：<br>
1.永豐金證券帳號：登入、取得報價<br>
2.下單憑證：有永豐金證券帳號後，下載電腦版下單軟體、下載憑證(.pfx檔案)，並指定憑證檔案的路徑載入<br>

### 登入永豐帳號

In [1]:
import pandas as pd
key = pd.read_csv('key.csv')  
account = key.iloc[0,0]
password = key.iloc[0,1]

In [2]:
import shioaji as sj
api = sj.Shioaji()
accounts = api.login(account,password)

# 載入憑證(下單才需要)
# api.activate_ca(
#     ca_path="/c/your/ca/path/Sinopac.pfx",
#     ca_passwd="YOUR_CA_PASSWORD",
#     person_id="Person of this Ca",
# )

Response Code: 0 | Event Code: 0 | Info: host '203.66.91.161:80', hostname '203.66.91.161:80' IP 203.66.91.161:80 (host 1 of 1) (host connection attempt 1 of 1) (total connection attempt 1 of 1) | Event: Session up


### 取得日期

In [3]:
import datetime 
today = datetime.date.today()
yesterday = datetime.date.today()-datetime.timedelta(days=1)
print('yesterday = ',yesterday,'today = ',today)

yesterday =  2021-01-04 today =  2021-01-05


### 取得近兩日的分K資料

In [4]:
import pandas as pd
ID = 2303
stock_ID = str(ID)
contract = api.Contracts.Stocks[stock_ID]
kbars = api.kbars(contract, start=str(yesterday), end=str(today))
df = pd.DataFrame({**kbars})
df.ts = pd.to_datetime(df.ts)
df

,Open,Low,Volume,High,Close,ts
0,46.15,46.00,11413,46.45,46.35,2021-01-04 09:01:00
1,46.35,46.10,4701,46.50,46.20,2021-01-04 09:02:00
2,46.20,46.05,3887,46.25,46.20,2021-01-04 09:03:00
3,46.20,45.85,6900,46.20,45.90,2021-01-04 09:04:00
4,45.90,45.50,6267,45.90,45.50,2021-01-04 09:05:00
...,...,...,...,...,...,...
535,46.25,46.25,0,46.25,46.25,2021-01-05 13:26:00
536,46.25,46.25,0,46.25,46.25,2021-01-05 13:27:00
537,46.25,46.25,0,46.25,46.25,2021-01-05 13:28:00
538,46.25,46.25,0,46.25,46.25,2021-01-05 13:29:00


### 將分K收盤價轉成series

In [5]:
stock_price = pd.Series(df['Close'].values, index=df['ts']).tail(200)
stock_price

ts
2021-01-05 10:11:00    45.10
2021-01-05 10:12:00    45.25
2021-01-05 10:13:00    45.20
2021-01-05 10:14:00    45.20
2021-01-05 10:15:00    45.20
                       ...  
2021-01-05 13:26:00    46.25
2021-01-05 13:27:00    46.25
2021-01-05 13:28:00    46.25
2021-01-05 13:29:00    46.25
2021-01-05 13:30:00    46.35
Length: 200, dtype: float64

### 盤中取得最新的200根分K並丟進模型，每分鐘輸出即時預測結果

In [ ]:
import datetime
import schedule
import time
import keras
model = keras.models.load_model(stock_ID +'.h5')
time_period = 200

def CNN_prdiction_intraday():
    now_X = []
    
    snapshot = api.snapshots([contract])
    update_close = pd.DataFrame(snapshot)
    update_close.ts = pd.to_datetime(update_close.ts)
    update_close = pd.Series(update_close['close'].values, index=pd.Series(datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')))
    global stock_price
    stock_price = stock_price.append(update_close).tail(200)
    
    #正規化
    for i in range(len(stock_price) - time_period, len(stock_price) - (time_period-1)):
        P = stock_price.iloc[i:i+time_period].values  # 0~200-1
        now_X.append((P - P.min())/(P.max() - P.min()))

    now_X = np.array(now_X)
    now_X = now_X.reshape(len(now_X), time_period, 1)
    
    #將最新的200根分K丟進模型
    prediction_now = 1-(np.squeeze(model.predict(now_X)))
    
    #輸出即時預測結果
    print('time = ',datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
    print('prediction = ',round(prediction_now,4)*100,'%')

schedule.every().minute.at(":00").do(CNN_prdiction_intraday)  # 每分鐘執行一次，取得盤中最新的200根分K並丟進模型，輸出即時預測結果
while 1:
    schedule.run_pending()

time =  2021-01-05 14:18:00
prediction =  0.41000000000000003 %
time =  2021-01-05 14:18:00
prediction =  0.38 %
time =  2021-01-05 14:19:00
prediction =  0.41000000000000003 %
time =  2021-01-05 14:19:00
prediction =  0.38999999999999996 %
